In [1]:
import sys
import os



import os
import csv
import torch
import pandas as pd
import matplotlib.pyplot as plt
import random
from torch.utils.data import TensorDataset, DataLoader
import pickle
from gpolnel.problems.inductive_programming import SML
from gpolnel.utils.utils import train_test_split
from gpolnel.utils.ffunctions import Ffunctions
from gpolnel.utils.inductive_programming import function_map
from gpolnel.algorithms.genetic_algorithm import GeneticAlgorithm
from gpolnel.operators.initializers import grow, prm_grow, ERC
from gpolnel.operators.variators import swap_xo, prm_subtree_mtn
from gpolnel.operators.selectors import prm_tournament, roulette_wheel, rank_selection
from gpolnel.utils.inductive_programming import _execute_tree
seed=1
random.seed(seed)
import numpy as np
parent_dir = os.path.abspath(os.path.join(os.path.dirname('../NEUROEVOLUTION'), '..'))
if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)

from utils import kfold
from utils import drop_features
from utils import scale
from utils import clipp
from utils import cv_logger_init
from utils import cv_logger_append
from utils import device
#_evaluate_individual_ffunction

ModuleNotFoundError: No module named 'gpolnel'

In [ ]:
# from sklearn.metrics import mean_squared_error

# def evaluate_solution(mheuristic, X_test, y_test):
#     y_pred = np.float(_execute_tree(mheuristic.best_sol.repr_, X_test))
#     return mean_squared_error(y_pred, y_test)

In [ ]:
# X_train = torch.tensor(pd.read_csv('datamart/X_train_clipped.csv').values)
# X_test = torch.tensor(pd.read_csv('datamart/X_test_clipped.csv').values)
# y_train = torch.tensor(pd.read_csv('datamart/y_lactose_train.csv')['lactose_percent'].values)
# y_test = torch.tensor(pd.read_csv('datamart/y_lactose_test.csv')['lactose_percent'].values)
X = pd.read_csv('datamart/data_project_nel.csv')
y = pd.read_csv('datamart/y_lactose.csv')['lactose_percent']
X = drop_features(X)

In [ ]:
fset = [function_map['add'],  function_map['sub']]#function_map['sub'],
total_batches = 1
batch_size = X.shape[0]
sspace_sml = {
    'n_dims': X.shape[1],
    'function_set': fset, 'constant_set': ERC(-5., 5.),
    'p_constants': 0.1,
    'max_init_depth': 3,
    'max_depth': 5, 
    'n_batches': total_batches,
    'device': device
}

ps =  100
selection_pressure = .1
mutation_prob = .7
xo_prob = .9
has_elitism = True
allow_reproduction = False
ffunction = Ffunctions('rmse')



In [ ]:
shuffle= True
total_batches = 1
batch_size = X.shape[0]
# Creates training and validatation data sets
# X = torch.tensor(X.values)
# y = torch.tensor(y.values)
# ds_train = TensorDataset(X, y)
# ds_test = TensorDataset(X, y)

# Creates training and test data loaders
# dl_train = DataLoader(ds_train, batch_size, shuffle)
# dl_test = DataLoader(ds_test, batch_size, shuffle)

In [ ]:
def gp_train(dl_train, dl_val, ps, sspace_sml, ffunction, log_path, selection_pressure, mutation_prob, xo_prob, has_elitism, allow_reproduction, seed=1, n_iter=3000, device='cpu',save_model=False, log=1, id=None):

    pi_sml = SML(
        sspace=sspace_sml,
        ffunction=ffunction,
        dl_train=dl_train, dl_test=dl_val,  # For the algorithm, the unseen is our validation!
        n_jobs=8
    )

    mheuristic = GeneticAlgorithm(
        pi=pi_sml,
        initializer=grow,
        selector=prm_tournament(pressure=selection_pressure),  #prm_tournament(pressure=selection_pressure)
        crossover=swap_xo,
        mutator=prm_subtree_mtn(initializer=prm_grow(sspace_sml)),
        pop_size=ps,
        p_m=mutation_prob,
        p_c=xo_prob,
        elitism=has_elitism,
        reproduction=allow_reproduction,  # False = or xo or mutation
        device=device,
        seed=seed
    )
    mheuristic._initialize()

    mheuristic.solve(
        n_iter,
        verbose=0, log=log, log_path=log_path,
        test_elite=True
    )
    
    
    return mheuristic

In [ ]:
# m = gp_train(dl_train = dl_train, dl_val = dl_test, ps=ps, sspace_sml = sspace_sml, ffunction=ffunction, log_path='abc.csv',  selection_pressure=0.05, mutation_prob=0.2, xo_prob=0.8, has_elitism=True, allow_reproduction=True, seed=1, n_iter=30, device='cpu')
# m.best_sol.printTree()
# m.best_sol.fit

In [ ]:
def gp_cross_validation(X, y, kf, ps, sspace_sml, ffunction, log_path_cv, log_path_train,  selection_pressure, mutation_prob, xo_prob, has_elitism, allow_reproduction, seed, n_iter, device, id=None, log=False):
    results = []
    feature_names = X.columns
    
    if log:
        cv_logger_init(log_path_cv)
            
    for fold, (train_index, val_index) in enumerate(kf.split(X)):
        print(type(X))
        X_train, X_val = X.loc[train_index], X.loc[val_index]
        y_train, y_val = y.loc[train_index], y.loc[val_index]
        
        print(type(X_train))
        X_train, X_val = clipp(X_train, X_val)
        X_train, X_val = scale(X_train, X_val)
        
        ds_train = TensorDataset(torch.tensor(X_train.values), torch.tensor(y_train.values))
        ds_val = TensorDataset(torch.tensor(X_val.values), torch.tensor(y_val.values))

        # Creates training and test data loaders
        dl_train = DataLoader(ds_train, batch_size, shuffle)
        dl_val = DataLoader(ds_val, batch_size, shuffle)

        m = gp_train(dl_train = dl_train, dl_val = dl_val, ps=ps, sspace_sml = sspace_sml, ffunction=ffunction, log_path=log_path_train,  selection_pressure=selection_pressure, mutation_prob=mutation_prob, xo_prob=xo_prob, has_elitism=has_elitism, allow_reproduction=allow_reproduction,  seed=seed, n_iter=n_iter, device=device)
        m.best_sol.printTree(feature_names=feature_names)
        score = m.best_sol.fit
        results.append(score)
    
        if log:
            cv_logger_append(log_path_cv, id, fold, score)
        
    avg_score = np.mean(results)
    print('cv_score:', avg_score)
    
    return avg_score

In [ ]:
# gp_cross_validation(X, y, kfold, ps=ps, sspace_sml = sspace_sml, ffunction = ffunction, log_path_cv='abc.csv', log_path_train='ttt.csv',  selection_pressure=0.05, mutation_prob=0.2, xo_prob=0.8, has_elitism=True, allow_reproduction=True, seed=1, n_iter=10, device='cpu', log=True)

In [ ]:
fsets = [function_map['add'],  function_map['mul'], function_map['div'],function_map['sub']] #,  function_map['mean']]
constant_sets = [ERC(-1., 1.), ERC(-5., 5.), ERC(-10., 10.)]
pss = [50, 100, 250, 500, 1000]
p_constantss = [0.05, 0.1, 0.2, 0.3]
max_init_depths = [2, 3, 4, 5, 6]
max_depths = [5, 6, 7, 8, 9]
selection_pressures = [0.05, 0.1, 0.2, 0.3, 0.4]
mutatuion_probs = [0.05, 0.1, 0.15, 0.2, 0.5]
xo_probs = [0.05, 0.1, 0.15, 0.2, 0.5]
has_elitisms = [True, False]
allow_reproductions = [True, False]
n_iters = [50, 100, 250, 500, 1000]
ffunction = Ffunctions('rmse')

best_score = np.inf

name = 'logs/GP/' + 'random_search_total' + '.csv'
with open(name, 'w', newline='\n') as csvfile:
    w = csv.writer(csvfile, delimiter=';')
    w.writerow(['id']+['fset']+['constant_set']+['ps']+['p_constants']+['max_init_depth']+['max_depth']+['selection_pressure']+['mutation_prob']+['xo_prob']+['has_elitism']+['allow_reproduction']+['n_iter']+['score'])


for i in range(200):

    total_batches = 1
    batch_size = X.shape[0]
    
    fset = random.sample(fsets, random.randint(1, len(fsets)))
    print(fset)
    constant_set = random.choice(constant_sets)
    ps = random.choice(pss)
    p_constants = random.choice(p_constantss)
    max_init_depth = random.choice(max_init_depths)
    max_depth = random.choice(max_depths)
    selection_pressure = random.choice(selection_pressures)
    mutation_prob = random.choice(mutatuion_probs)
    xo_prob = random.choice(xo_probs)
    has_elitism = random.choice(has_elitisms)
    allow_reproduction = random.choice(allow_reproductions)
    n_iter = random.choice(n_iters)
    log_path_cv = 'logs/GP/' + f'{i}' + '_cv_results.csv'
    log_path_train = 'logs/GP/' + f'{i}' + '_cv_train.csv'

    sspace_sml = {
        'n_dims': X.shape[1],
        'function_set': fset, 'constant_set': constant_set,
        'p_constants': p_constants,
        'max_init_depth': max_init_depth,
        'max_depth': max_depth, 
        'n_batches': total_batches,
        'device': device
    }
    
    score = gp_cross_validation(X, y, kfold, ps=ps, sspace_sml = sspace_sml, ffunction=ffunction, log_path_cv=log_path_cv, log_path_train=log_path_train,  selection_pressure=selection_pressure, mutation_prob=mutation_prob, xo_prob=xo_prob, has_elitism=has_elitism, allow_reproduction=allow_reproduction, seed=1, n_iter=n_iter, device='cpu')
    print(score)
    
    name = 'logs/GP/' + 'random_search_total' + '.csv'
    with open(name, 'a', newline='\n') as csvfile:
        w = csv.writer(csvfile, delimiter=';')
        w.writerow([i]+[fset]+[constant_set]+[ps]+[p_constants]+[max_init_depth]+[max_depth]+[selection_pressure]+[mutation_prob]+[xo_prob]+[has_elitism]+[allow_reproduction]+[n_iter]+[score])
    
    if score < best_score:
        best_score = score
        print('Iteration '+ str(i))
        print(f'New best validation score: {best_score}')
        best_params = [fset, constant_set, ps, p_constants, max_init_depth, max_depth, selection_pressure, mutation_prob, xo_prob, has_elitism, allow_reproduction, n_iter]
        
        

[sub]
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
4.8504
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
4.8504
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
4.8504
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
4.5296
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
sub( sub( rumination_min_day, rumination_min_day ) -4.8707 )
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
4.9277
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
sub( sub( high_cdt_by_100_milkings, high_cdt_by_100_milkings ) -4.8707 )
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
4.5833
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
4.9277
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
4.5296
cv_score: 0.15679488
0.15679488
Iteration 0


KeyboardInterrupt: 

In [ ]:
best_params

[[sub],
 100,
 0.05,
 4,
 5,
 0.05,
 0.05,
 0.5,
 True,
 False,
 100]